In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ax.core import (
    SearchSpace,
    RangeParameter,
    ChoiceParameter,
    ParameterType,
)
from ax.generation_strategy.generation_strategy import GenerationStrategy
from ax.generation_strategy.generation_node import GenerationNode
from ax.generation_strategy.generator_spec import GeneratorSpec
from ax_cbo import ContextualBayesOptAx

from sheets_api import pullData


ModuleNotFoundError: No module named 'ax_cbo'

In [ ]:
def build_search_space():
    """
    Returns parameter search space in the context of SLA printing with Rayware
    """
    search_space = SearchSpace(
        parameters=[
            ChoiceParameter(
                name="layer_thickness_um",
                parameter_type=ParameterType.INT,
                values=[50, 100],
                is_ordered=True,
                sort_values=True,
            ),
            ChoiceParameter(
                name="fit_adjustment",
                parameter_type=ParameterType.INT,
                values=[-250, -150, -50, 0, 50, 150, 250],
                is_ordered=True,
                sort_values=True,
            ),
            RangeParameter(
                name="z_rotation_deg",
                parameter_type=ParameterType.FLOAT,  # could be changed to int
                lower=0.0,
                upper=90.0,
            ),
        ]
    )
    return search_space


def load_dataset(is_testing: bool):
    """
    Returns Dataframe from Google Spreadsheet or fake dataset

    Args:
        is_testing: bool
           Uses fake dataset when True, Google Spreadsheet when False
    """
    if is_testing:
        return pd.read_csv("./datasets/dataset.csv")

    return pullData(verbose=True)


def context_increase_test(c_new, strength=2):
    for value in c_new.values():
        value += strength

    return c_new


def experiment(cbo, c_new):
    # initilaize historical data
    df = load_dataset(is_testing=True)  # pulling fake data for now
    cbo.add_historical(df)
    print("Loaded Dataset into CBO surrogate")

    suggested = cbo.suggest(isOnline=True, c_t=c_new)  # online loop for testing
    trial = suggested["trial"]
    # params = suggested["params"]
    print(trial)
    return trial